# Data Science Project - Part 1 : Data Collection

In this Notebook, we will collect texts from different categories in Wikipedia:
- biographies of [US Presidents](https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States) 
- biographies of [Scientists Women in the 21st century](https://en.wikipedia.org/wiki/List_of_female_scientists_in_the_21st_century)